# Section 1: Web scraping

### Importation des Bibliothèques

###### Pour commencer notre projet de web scraping, nous devons importer les bibliothèques nécessaires. Ces outils nous permettront de récupérer les pages web, d'extraire les informations utiles et de structurer les données pour une analyse plus facile.

1)  requests : Utilisée pour envoyer des requêtes HTTP. Elle nous permet d'accéder aux pages web que nous voulons scraper.
2) BeautifulSoup : Cette bibliothèque aide à analyser et à extraire des données des fichiers HTML et XML. Elle rend la navigation et la recherche d'éléments spécifiques sur une page web plus simples.
3) pandas : Une bibliothèque puissante pour la manipulation et l'analyse des données. Elle nous permet de créer des DataFrames, qui sont des structures de données tabulaires similaires aux tables de bases de données ou aux feuilles de calcul Excel.

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

###### Mubawab.ma est un site web marocain spécialisé dans l'immobilier. Il permet aux utilisateurs de rechercher, vendre et louer des biens immobiliers tels que des appartements, des maisons et des terrains au Maroc. Le site offre une plateforme où les utilisateurs peuvent publier des annonces immobilières et rechercher des propriétés disponibles dans différentes villes du Maroc.

1) On définit l'URL du site web à scraper dans la variable Website.
2) On utilise requests pour récupérer le contenu HTML de la page avec une requête GET.
3) Ce contenu est ensuite analysé par BeautifulSoup pour faciliter l'extraction des données plus tard.

In [5]:
# Définition de l'URL du site web que nous allons scraper
Website = "https://www.mubawab.ma/"

# Envoi d'une requête HTTP GET à l'URL spécifiée et récupération du contenu de la page web
html_content = requests.get(Website).content

# Analyse du contenu HTML à l'aide de BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

#### Extraction et affichage des types de vente

In [6]:
# Recherche de l'élément <ul> avec l'ID 'scat-sale' dans le contenu HTML analysé par BeautifulSoup
type_ul_element = soup.find('ul', id='scat-sale')

# Création d'une liste des textes de tous les éléments <li> à l'intérieur de l'élément <ul> trouvé
type_list = [li.get_text(strip=True) for li in type_ul_element.find_all('li')]

# Affichage de la liste des types de vente récupérés
print(type_list)


['Appartements', 'Terrains', 'Villas', 'Maisons', 'Locaux commerciaux', 'Riads', 'Bureaux']


#### Extraction et affichage des noms de villes

In [7]:
# Recherche de l'élément <ul> avec l'ID 'cit-apartment-sale' dans le contenu HTML analysé par BeautifulSoup
city_ul_element = soup.find('ul', id='cit-apartment-sale')

# Création d'une liste des textes de tous les éléments <a> à l'intérieur de l'élément <ul> trouvé
city_list = [a.get_text(strip=True) for a in city_ul_element.find_all('a')]

# Affichage de la liste des villes récupérées
print(city_list)


['Casablanca', 'Marrakech', 'Tanger', 'Rabat', 'Agadir', 'Kénitra', 'Fès', 'Salé', 'Mohammedia', 'Temara']


#### Conversion des noms de ville en minuscules et affichage de la liste résultante.

In [8]:
# Convertir chaque nom de ville en minuscules
city_list_lower = [city.lower() for city in city_list]

# Afficher la liste des noms de ville en minuscules
print(city_list_lower)


['casablanca', 'marrakech', 'tanger', 'rabat', 'agadir', 'kénitra', 'fès', 'salé', 'mohammedia', 'temara']


In [7]:
def get_city_link(city):
    """
    Génère une URL pour récupérer les annonces d'appartements basée sur le nom de la ville.

    Args:
    - city (str): Nom de la ville.

    Returns:
    - url (str): URL générée pour les annonces d'appartements à vendre dans la ville spécifiée.
    """
    url = "https://www.mubawab.ma/fr/st/" + city.lower().replace(" ", "-") + "/appartements-a-vendre"
    return url



In [8]:

def get_city_page_link(city, page):
    """
    Génère une URL pour une page spécifique des annonces d'appartements pour une ville donnée.

    Args:
    - city (str): Nom de la ville.
    - page (int): Numéro de la page des annonces.

    Returns:
    - url (str): URL générée pour la page spécifique des annonces d'appartements à vendre dans la ville.
    """
    url = f"https://www.mubawab.ma/fr/st/{city}/appartements-a-vendre:p:{page}"
    return url


##### Exemple d'utilisation de la fonction get_city_page_link

In [9]:
for city in city_list_lower:
    print(get_city_link(city))

https://www.mubawab.ma/fr/st/casablanca/appartements-a-vendre
https://www.mubawab.ma/fr/st/marrakech/appartements-a-vendre
https://www.mubawab.ma/fr/st/tanger/appartements-a-vendre
https://www.mubawab.ma/fr/st/rabat/appartements-a-vendre
https://www.mubawab.ma/fr/st/agadir/appartements-a-vendre
https://www.mubawab.ma/fr/st/kénitra/appartements-a-vendre
https://www.mubawab.ma/fr/st/fès/appartements-a-vendre
https://www.mubawab.ma/fr/st/salé/appartements-a-vendre
https://www.mubawab.ma/fr/st/mohammedia/appartements-a-vendre
https://www.mubawab.ma/fr/st/temara/appartements-a-vendre


In [10]:
def get_listing_links(url):
    # Récupère les liens des annonces à partir d'une URL spécifiée.
    # Args:
    # - url (str): L'URL à analyser pour extraire les liens des annonces.
    # Returns:
    # - links (list): Une liste d'URL correspondant aux annonces immobilières trouvées sur la page.
    
    # Envoyer une requête GET à l'URL spécifiée
    response = requests.get(url)

    # Extraire le contenu HTML brut
    html_content = response.content

    # Analyser le HTML à l'aide de BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Trouver toutes les boîtes d'annonces sur la page
    listing_boxes = soup.find_all('li', class_='listingBox')

    # Extraire les liens de chaque boîte d'annonce
    links = [listing_box['linkref'] for listing_box in listing_boxes if 'linkref' in listing_box.attrs]

    return links




##### Exemple d'utilisation de la fonction get_listing_links

In [11]:
url = 'https://www.mubawab.ma/fr/st/marrakech/appartements-a-vendre'
listing_links = get_listing_links(url)
for link in listing_links:
    print(link)

https://www.mubawab.ma/fr/pa/7708910/appartement-de-83m%C2%B2-%C3%A0-vendre-jnane-el-ali-gu%C3%A8liz
https://www.mubawab.ma/fr/pa/7915640/appartement-%C3%A0-l-achat-%C3%A0-gu%C3%A9liz-superficie-64-m%C2%B2
https://www.mubawab.ma/fr/pa/7662105/appartement-premium-%C3%A0-vendre-%C3%A0-jnane-el-ali-gu%C3%A8liz
https://www.mubawab.ma/fr/pa/7856308/vend-appartement-%C3%A0-majorelle-superficie-73-m%C2%B2
https://www.mubawab.ma/fr/pa/7906280/appartement-%C3%A0-l-achat-%C3%A0-hivernage-surface-totale-88-m%C2%B2
https://www.mubawab.ma/fr/pa/7710244/appartement-de-86m%C2%B2-%C3%A0-vendre-%C3%A0-gu%C3%A8liz
https://www.mubawab.ma/fr/pa/7906278/bel-appartement-%C3%A0-vendre-%C3%A0-hivernage-superficie-60-m%C2%B2
https://www.mubawab.ma/fr/pa/7837804/superbe-appartement-%C3%A0-vendre-%C3%A0-samlalia-superficie-46-m%C2%B2
https://www.mubawab.ma/fr/p/3858/locaux-commerciaux-et-plateaux-bureaux
https://www.mubawab.ma/fr/pa/7915667/appartement-en-vente-%C3%A0-gu%C3%A9liz-1-chambre
https://www.mubawab.ma

In [12]:
def get_number_of_pages(soup):
    # Fonction qui extrait le nombre de pages à partir d'un objet BeautifulSoup.
    # Args:
    # - soup (BeautifulSoup object): Objet BeautifulSoup représentant le contenu HTML analysé.
    # Returns:
    # - pages_number (int): Nombre de pages extrait de l'élément spécifié.

    # Trouver l'élément contenant les informations sur les pages
    page_info = soup.find('p', class_='fSize11 centered')

    # Obtenir le texte de cet élément
    pages_text = page_info.get_text(strip=True)

    # Extraire le nombre de pages à partir du texte
    pages_number = int(pages_text.split('|')[-1].strip().split()[0].split('-')[-1])

    return pages_number




##### Exemple d'utilisation de la fonction get_number_of_pages

In [13]:

url = 'https://www.mubawab.ma/fr/st/marrakech/appartements-a-vendre'
html_content = requests.get(url).content
soup = BeautifulSoup(html_content, "html.parser")
num_pages = get_number_of_pages(soup)

print(f"Number of pages: {num_pages}")


Number of pages: 58


#### définition de certaines fonctions pour les donnée de chaque appartement

In [ ]:
def get_price(soup):
    # Fonction qui extrait le prix d'une annonce à partir d'un objet BeautifulSoup.
    # Args:
    # - soup (BeautifulSoup object): Objet BeautifulSoup représentant le contenu HTML analysé.
    # Returns:
    # - price (float or None): Prix de l'annonce extrait, ou None si aucun prix n'est trouvé.

    # Trouver l'élément contenant le prix
    price_elem = soup.find('h3', class_='orangeTit')

    # Vérifier si un prix a été trouvé
    if price_elem:
        # Extraire le texte du prix, nettoyer et convertir en float
        price = price_elem.get_text().strip().replace('\xa0', '').replace('DH', '').strip()
        return float(price)
    
    # Retourner None si aucun prix n'est trouvé
    return None


In [ ]:
def get_area(soup):
    # Fonction qui extrait la surface d'une annonce à partir d'un objet BeautifulSoup.
    # Args:
    # - soup (BeautifulSoup object): Objet BeautifulSoup représentant le contenu HTML analysé.
    # Returns:
    # - area (float or None): Surface de l'annonce extrait, ou None si aucune surface n'est trouvée.

    # Parcourir tous les éléments <div> avec la classe 'adDetailFeature'
    for div in soup.find_all('div', class_='adDetailFeature'):
        # Trouver le premier élément <span> dans le <div>
        span = div.find('span')

        # Vérifier si un <span> a été trouvé et s'il contient 'm²' dans son texte
        if span and 'm²' in span.get_text(strip=True):
            # Extraire le texte du <span>, nettoyer et convertir en float
            area = span.get_text(strip=True).replace('m²', '').strip()
            return float(area)
    
    # Retourner None si aucune surface n'est trouvée
    return None


In [ ]:
def get_rooms(soup):
    # Fonction qui extrait le nombre de chambres d'une annonce à partir d'un objet BeautifulSoup.
    # Args:
    # - soup (BeautifulSoup object): Objet BeautifulSoup représentant le contenu HTML analysé.
    # Returns:
    # - rooms (int or None): Nombre de chambres extrait de l'annonce, ou None si aucun nombre de chambres n'est trouvé.

    # Parcourir tous les éléments <div> avec la classe 'adDetailFeature'
    for div in soup.find_all('div', class_='adDetailFeature'):
        # Trouver le premier élément <span> dans le <div>
        span = div.find('span')

        # Vérifier si un <span> a été trouvé et s'il contient 'Chambres' ou 'Chambre' dans son texte
        if span and ('Chambres' in span.get_text(strip=True) or 'Chambre' in span.get_text(strip=True)):
            # Extraire le texte du <span>, nettoyer et convertir en entier
            rooms_text = span.get_text(strip=True).replace('Chambres', '').replace('Chambre', '').strip()
            return int(rooms_text)
    
    # Retourner None si aucun nombre de chambres n'est trouvé
    return None


In [ ]:
def get_bathrooms(soup):
    # Fonction qui extrait le nombre de salles de bains d'une annonce à partir d'un objet BeautifulSoup.
    # Args:
    # - soup (BeautifulSoup object): Objet BeautifulSoup représentant le contenu HTML analysé.
    # Returns:
    # - bathrooms (int or None): Nombre de salles de bains extrait de l'annonce, ou None si aucun nombre de salles de bains n'est trouvé.

    # Parcourir tous les éléments <div> avec la classe 'adDetailFeature'
    for div in soup.find_all('div', class_='adDetailFeature'):
        # Trouver le premier élément <span> dans le <div>
        span = div.find('span')

        # Vérifier si un <span> a été trouvé et s'il contient 'Salles de bains' ou 'Salle de bain' dans son texte
        if span and ('Salles de bains' in span.get_text(strip=True) or 'Salle de bain' in span.get_text(strip=True)):
            # Extraire le texte du <span>, nettoyer et convertir en entier
            bathrooms_text = span.get_text(strip=True).replace('Salles de bains', '').replace('Salle de bain', '').strip()
            return int(bathrooms_text)
    
    # Retourner None si aucun nombre de salles de bains n'est trouvé
    return None


In [ ]:
def get_arrondissement(soup):
    # Fonction qui extrait l'arrondissement d'une annonce à partir d'un objet BeautifulSoup.
    # Args:
    # - soup (BeautifulSoup object): Objet BeautifulSoup représentant le contenu HTML analysé.
    # Returns:
    # - arrondissement (str or None): Arrondissement extrait de l'annonce, ou None si aucun arrondissement n'est trouvé.

    # Trouver l'élément <h3> avec la classe 'greyTit'
    h3 = soup.find('h3', class_='greyTit')

    # Vérifier si un <h3> a été trouvé
    if h3:
        # Obtenir le texte de cet élément
        text = h3.get_text(strip=True)
        
        # Extraire l'arrondissement à partir du texte
        arrondissement = text.split(' à')[0].strip()
        return arrondissement
    
    # Retourner None si aucun arrondissement n'est trouvé
    return None


##### Exemple d'utilisation de toutes les fonctions

In [ ]:
# Récupérer le contenu HTML de la page de l'appartement
url = "https://www.mubawab.ma/fr/pa/7659737/vente-d-un-bel-appartement-%C3%A0-yacoub-al-mansour-3-chambres-agr%C3%A9ables"
html_content = requests.get(url).content
soup = BeautifulSoup(html_content, "html.parser")

# Extraire les détails de l'appartement
price = get_price(soup)
area = get_area(soup)
rooms = get_rooms(soup)
bathrooms = get_bathrooms(soup)
arrondissement = get_arrondissement(soup)

# Afficher les informations extraites
print(f"Prix : {price} DH")
print(f"Surface : {area} m²")
print(f"Pièces : {rooms}")
print(f"Salle de bains : {bathrooms}")
print(f"Arrondissement : {arrondissement}")


#### cette partie du code permettra d'extraire toutes les annonces d'appartements dans chaque ville, mais cela prendra beaucoup de temps. 

In [15]:

# Initialize an empty list to store the house details
houses = []

for city in city_list_lower:
    print(f"Scraping listings for city: {city.capitalize()}")
    house_count = 0
    first_page_url = get_city_link(city)
    response = requests.get(first_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Get the number of pages
    try:
        number_of_pages = get_number_of_pages(soup)
    except Exception as e:
        print(f"Error getting number of pages for {city}: {e}")
        number_of_pages = 1
    
    # Iterate over the pages
    for page in range(1, number_of_pages + 1):
        if page == 1:
            city_page_url = first_page_url
        else:
            city_page_url = get_city_page_link(city, page)
        
        listing_links = get_listing_links(city_page_url)
        
        for house_url in listing_links:
            try:
                response = requests.get(house_url)
                soup = BeautifulSoup(response.content, 'html.parser')
                
                area = get_area(soup)
                rooms = get_rooms(soup)
                bathrooms = get_bathrooms(soup)
                arrondissement = get_arrondissement(soup)
                price = get_price(soup)
                
                houses.append({
                    'city': city,
                    'arrondissement': arrondissement,
                    'price' : price,
                    'area': area,
                    'rooms': rooms,
                    'bathrooms': bathrooms,
                    'url': house_url
                })
                
                house_count += 1
                print(f'Houses scraped :{house_count}')
            except Exception as e:
                print(f"Error processing {house_url}: {e}")
    
    print(f"Total houses scraped for {city.capitalize()}: {house_count}")



Scraping listings for city: Casablanca
Houses scraped :1


KeyboardInterrupt: 

#### C'est pourquoi nous allons utiliser cette partie à la place, car elle contient une variable num_houses_to_scrape qui nous oblige à récupérer exactement un nombre prédéfini d'annonces dans chaque ville.

In [20]:
# Définir le nombre de maisons à collecter par ville
num_houses_to_scrape = 70

# Initialiser une liste vide pour stocker les détails des maisons
houses = []

# Parcourir chaque ville dans la liste des villes converties en minuscules
for city in city_list_lower:
    print(f"Scraping listings for city: {city.capitalize()}")
    house_count = 0
    
    # Obtenir l'URL de la première page des annonces pour cette ville
    first_page_url = get_city_link(city)
    response = requests.get(first_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Récupérer le nombre de pages d'annonces pour cette ville
    try:
        number_of_pages = get_number_of_pages(soup)
    except Exception as e:
        print(f"Error getting number of pages for {city}: {e}")
        number_of_pages = 1
    
    # Parcourir les pages d'annonces
    for page in range(1, number_of_pages + 1):
        if house_count >= num_houses_to_scrape:
            break
        
        # Construire l'URL de la page actuelle d'annonces pour cette ville
        if page == 1:
            city_page_url = first_page_url
        else:
            city_page_url = get_city_page_link(city, page)
        
        # Obtenir les liens des annonces sur cette page
        listing_links = get_listing_links(city_page_url)
        
        # Parcourir chaque lien d'annonce
        for house_url in listing_links:
            if house_count >= num_houses_to_scrape:
                break
            
            try:
                # Accéder à l'URL de l'annonce et analyser le contenu HTML
                response = requests.get(house_url)
                soup = BeautifulSoup(response.content, 'html.parser')
                
                # Extraire les informations spécifiques de l'annonce
                area = get_area(soup)
                rooms = get_rooms(soup)
                bathrooms = get_bathrooms(soup)
                arrondissement = get_arrondissement(soup)
                price = get_price(soup)
                
                # Ajouter les détails de l'annonce à la liste 'houses'
                houses.append({
                    'city': city,
                    'arrondissement': arrondissement,
                    'price' : price,
                    'area': area,
                    'rooms': rooms,
                    'bathrooms': bathrooms,
                    'url': house_url
                })
                
                # Augmenter le compteur de maisons collectées
                house_count += 1
                print(f'Houses scraped: {house_count}')
            except Exception as e:
                print(f"Error processing {house_url}: {e}")
    
    # Afficher le nombre total de maisons collectées pour cette ville
    print(f"Total houses scraped for {city.capitalize()}: {house_count}")


Scraping listings for city: Casablanca
Houses scraped: 1
Houses scraped: 2
Houses scraped: 3
Houses scraped: 4
Houses scraped: 5
Houses scraped: 6
Houses scraped: 7
Houses scraped: 8
Houses scraped: 9
Houses scraped: 10
Houses scraped: 11
Houses scraped: 12
Houses scraped: 13
Houses scraped: 14
Houses scraped: 15
Houses scraped: 16
Houses scraped: 17
Houses scraped: 18
Houses scraped: 19
Houses scraped: 20
Houses scraped: 21
Houses scraped: 22
Houses scraped: 23
Houses scraped: 24
Houses scraped: 25
Houses scraped: 26
Houses scraped: 27
Houses scraped: 28
Houses scraped: 29
Houses scraped: 30
Houses scraped: 31
Houses scraped: 32
Houses scraped: 33
Houses scraped: 34
Houses scraped: 35
Houses scraped: 36
Houses scraped: 37
Houses scraped: 38
Houses scraped: 39
Houses scraped: 40
Houses scraped: 41
Houses scraped: 42
Houses scraped: 43
Houses scraped: 44
Houses scraped: 45
Houses scraped: 46
Houses scraped: 47
Houses scraped: 48
Houses scraped: 49
Houses scraped: 50
Houses scraped: 51
H

In [40]:
# Convertir la liste de maisons en un DataFrame pandas
df = pd.DataFrame(houses)

# Afficher les 20 premières lignes du DataFrame
df.head(20)


,city,arrondissement,price,area,rooms,bathrooms,url
0,casablanca,Oulfa,1300000.0,101.0,3.0,2.0,https://www.mubawab.ma/fr/pa/7938271/apparteme...
1,casablanca,Oulfa,1300000.0,127.0,3.0,2.0,https://www.mubawab.ma/fr/pa/7938268/vend-appa...
2,casablanca,Oulfa,1110000.0,86.0,2.0,2.0,https://www.mubawab.ma/fr/pa/7938267/apparteme...
3,casablanca,Oulfa,1110000.0,86.0,2.0,2.0,https://www.mubawab.ma/fr/pa/7938266/vend-appa...
4,casablanca,Oulfa,1110000.0,100.0,2.0,2.0,https://www.mubawab.ma/fr/pa/7938265/bel-appar...
5,casablanca,Oulfa,1300000.0,97.0,3.0,2.0,https://www.mubawab.ma/fr/pa/7937019/superbe-a...
6,casablanca,Oulfa,1110000.0,81.0,2.0,2.0,https://www.mubawab.ma/fr/pa/7937016/vente-d-u...
7,casablanca,Californie,1689000.0,109.0,3.0,2.0,https://www.mubawab.ma/fr/pa/7902995/vend-appa...
8,casablanca,None,NaN,NaN,NaN,NaN,https://www.mubawab.ma/fr/p/3274/r%C3%A9sidenc...
9,casablanca,Oulfa,1300000.0,101.0,3.0,2.0,https://www.mubawab.ma/fr/pa/7938270/apparteme...


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            700 non-null    object 
 1   arrondissement  678 non-null    object 
 2   price           678 non-null    float64
 3   area            677 non-null    float64
 4   rooms           678 non-null    float64
 5   bathrooms       678 non-null    float64
 6   url             700 non-null    object 
dtypes: float64(4), object(3)
memory usage: 38.4+ KB


In [41]:
df.shape

(700, 7)

In [42]:
# Sauvegarder le DataFrame dans un fichier CSV
df.to_csv('houses.csv', index=False)

# Afficher un message indiquant que les données ont été sauvegardées
print("Les données ont été sauvegardées dans houses.csv")


Les données ont été sauvegardées dans houses.csv


# Section 2: Machine learning model


In [1]:

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score 



KeyboardInterrupt: 

#### 1) StandardScaler (sklearn.preprocessing)

**Description** : Standardise les caractéristiques en supprimant la moyenne et en mettant à l'échelle par écart-type unitaire.  
**Utilisation :** Normaliser les données pour améliorer les performances des algorithmes.

#### 2) OneHotEncoder (sklearn.preprocessing)

**Description :** Transforme les variables catégorielles en variables binaires.  
**Utilisation :** Convertir les catégories en une représentation numérique.

#### 3) train_test_split (sklearn.model_selection)

**Description :** Divise les données en ensembles d'entraînement et de test.  
**Utilisation :** Évaluer les performances du modèle.

#### 4) GridSearchCV (sklearn.model_selection)

**Description :** Recherche des meilleurs paramètres pour un modèle en utilisant la validation croisée.  
**Utilisation :** Optimiser les hyperparamètres.

#### 5) LinearRegression (sklearn.linear_model)

**Description :** Modèle de régression linéaire.  
**Utilisation :** Modéliser la relation entre variables dépendantes et indépendantes.

#### 6) RandomForestRegressor (sklearn.ensemble)

**Description :** Modèle de régression basé sur une forêt d'arbres de décision.  
**Utilisation :** Réaliser des prédictions robustes et réduire le surapprentissage.

#### 7) mean_squared_error (sklearn.metrics)

**Description :** Calcule l'erreur quadratique moyenne entre les valeurs prédites et réelles.  
**Utilisation :** Mesurer la précision d'un modèle.

#### 8) r2_score (sklearn.metrics)

**Description :** Calcule le coefficient de détermination \( R^2 \).  
**Utilisation :** Évaluer la proportion de variance expliquée par le modèle.


#### Chercher les valeurs nulles

In [ ]:
missing_values = df.isnull().sum()
print("\nMissing values in each column:")
print(missing_values)


Missing values in each column:
city               0
arrondissement    22
price             22
area              23
rooms             22
bathrooms         22
url                0
dtype: int64


#### Supprimer les valeurs nulles

In [ ]:

df = df.dropna(subset=['arrondissement', 'price', 'area', 'rooms', 'bathrooms'])


#### Vérification

In [ ]:
missing_values = df.isnull().sum()
print("\nMissing values in each column:")
print(missing_values)


Missing values in each column:
city              0
arrondissement    0
price             0
area              0
rooms             0
bathrooms         0
url               0
dtype: int64


#### Encodage des variables catégorielles.

In [ ]:

df = pd.get_dummies(df, columns=['city', 'arrondissement'], drop_first=True)


#### Séparation des caractéristiques et de la cible.

In [ ]:

X = df.drop(columns=['price', 'url'])
y = df['price']



#### Division des données et mise à l'échelle des caractéristiques.

In [ ]:
# Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



#### Fonction pour entraîner et évaluer des modèles (réutilisable pour différents modèles)

In [ ]:
def train_and_evaluate(model, X_train_scaled, X_test_scaled, y_train, y_test):
    # Entraîner le modèle sur les données d'entraînement
    model.fit(X_train_scaled, y_train)
    
    # Prédire les valeurs cibles sur les données de test
    y_pred = model.predict(X_test_scaled)
    
    # Calculer l'erreur quadratique moyenne entre les prédictions et les valeurs réelles
    mse = mean_squared_error(y_test, y_pred)
    
    # Calculer le coefficient de détermination (R²) pour évaluer la qualité des prédictions
    r2 = r2_score(y_test, y_pred)
    
    # Afficher le nom du modèle utilisé
    print(f'Modèle: {model.__class__.__name__}')
    
    # Afficher l'erreur quadratique moyenne
    print(f'Erreur quadratique moyenne: {mse}')
    
    # Afficher le coefficient de détermination (R²)
    print(f'R²: {r2}')
    
    # Retourner l'erreur quadratique moyenne et le coefficient de détermination (R²)
    return mse, r2




#### Calcul de l'erreur quadratique moyenne (MSE) et du coefficient de détermination (R2) pour une régression linéaire.

In [ ]:
# Train and Evaluate Linear Regression Model
mse_lr, r2_lr = train_and_evaluate(LinearRegression(), X_train_scaled, X_test_scaled, y_train, y_test)



Model: LinearRegression
Mean Squared Error: 9.376055293316795e+39
R-Squared: -4.96044374322648e+27


#### Optimisation des hyperparamètres pour le modèle Random Forest à l'aide de GridSearchCV.

In [ ]:
# Hyperparameter Tuning with GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42), param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best Parameters for Random Forest: {best_params}')
print(f'Best Score for Random Forest: {best_score}')



Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters for Random Forest: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Best Score for Random Forest: -4.233463292086127


#### Entraînement du modèle final Random Forest avec les meilleurs paramètres trouvés et évaluation.

In [ ]:
# Train the Final Random Forest Model with Best Parameters
final_model = RandomForestRegressor(**best_params, random_state=42)
final_model.fit(X_train_scaled, y_train)

# Evaluate the Final Model
mse_final, r2_final = train_and_evaluate(final_model, X_train_scaled, X_test_scaled, y_train, y_test)




Model: RandomForestRegressor
Mean Squared Error: 411913340600.00745
R-Squared: 0.7820754155975105


#### Finalement affichage des valeurs de l'erreur quadratique moyenne (MSE) et du coefficient de détermination (R-squared) pour la régression linéaire et le modèle Random Forest final.

In [ ]:
print(f'Linear Regression MSE: {mse_lr}')
print(f'Linear Regression R-Squared: {r2_lr}')
print(f'Final Random Forest MSE: {mse_final}')
print(f'Final Random Forest R-Squared: {r2_final}')


Linear Regression MSE: 9.376055293316795e+39
Linear Regression R-Squared: -4.96044374322648e+27
Final Random Forest MSE: 411913340600.00745
Final Random Forest R-Squared: 0.7820754155975105
